In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
#Подгружаем данные
data = pd.ExcelFile("GOS 2019 for AD.xlsx")
var_1 = data.parse('Вариант 1')
var_2 = data.parse('Вариант 2')
var_3 = data.parse('Вариант 3')
var_4 = data.parse('Вариант 4')

#### Поиск статистики правильных ответов

In [ ]:
# Ищем правильные ответы для каждого варианта путём сравнения ответа студента с соотетвующим верным ответом
# в колонке "Верные ответы" 
def func1(f):
    return f == var_1["Верный ответ"]
def func2(f):
    return f == var_2["Верный ответ"]
def func3(f):
    return f == var_3["Верный ответ"]
def func4(f):
    return f == var_4["Верный ответ"]
#Получаем 4 фрейма с числом правильных ответов для каждого студента 
true_1 = var_1.apply(func1).sum().to_frame().iloc[2:,:]
true_2 = var_2.apply(func2).sum().to_frame().iloc[2:,:]
true_3 = var_3.apply(func3).sum().to_frame().iloc[2:,:]
true_4 = var_4.apply(func4).sum().to_frame().iloc[2:,:]
#Получаем общую таблицу с числом правильных ответов для ВСЕХ студента 
true_all = pd.concat([true_1,true_2,true_3,true_4]).sort_values(by = [0]).rename_axis("Студент")
stat = true_all.rename(columns = {0: "Правильные ответы"}).reset_index()
stat.T.squeeze()



#### Поиск  рубежных  баллов  для квантильных оценок

In [ ]:
chisl = [1.5,3,4.5,6,7, 8] #выбираем числители соответсвующие единому знаменателю для отбора квантильных оценок
quants = []
for i in chisl:
       quants = np.append(quants,stat.quantile(i/9))
quant_values = np.append(np.append(8,np.round(quants,0)),41).astype(int) #последний элемент-41, так как необходимо будет,чтобы он в дальнейшим включался
quant_values

#### Узнаём какому баллу  соответсвует то или иное число верных ответов

In [ ]:
#Формируем диапазон баллов теста  для каждой из 10-ти бальны оценок 
suppoter = range(0,len(quant_values)-1)
span = []
for j in suppoter: 
    span.append(range(quant_values[j],quant_values[j+1]))

#Cоздаём фрейм, отражающий какому баллу по 10-ти бальной шкале соответсвует то или иное число верных ответов     
parts = []
suppoter2 = range(0,len(span))
for j in suppoter2:      
    df = pd.DataFrame({j+4: span[j]})
    parts.append(df)
scores = pd.concat(parts, ignore_index = True).stack().to_frame(name = ("Правильные ответы"))
scores.index.names = ['№',"Баллы"]
tab_scores = scores.reset_index(level = 1)

#### Присваем каждому студенту балл по 10-ти бальной школе в соответствии с количеством верных ответов 

In [ ]:
answers_scores = pd.merge(stat,tab_scores, on = "Правильные ответы")
#v=final.pivot( columns='Баллы', values='Студент')
answers_scores["Оценка"] = answers_scores["Баллы"]
final_scores = answers_scores.replace({"Оценка":{4:"Удовл",5:"Удовл",6:"Хорошо", 7:"Хорошо",8:"Отлично",9:"Отлично",10:"Отлично"}})
final_scores

## Удоволитворительно

In [ ]:
dd=final_scores[final_scores['Оценка'] == 'Удовл']
dd.set_index(['Студент', 'Баллы']).sort_index(level = 1)

## Хорошо

In [ ]:
dd=final_scores[final_scores['Оценка'] == 'Хорошо']
dd.set_index(['Студент', 'Баллы']).sort_index(level = 1)

## Отлично 

In [ ]:
dd=final_scores[final_scores['Оценка'] == 'Отлично']
dd.set_index(['Студент', 'Баллы']).sort_index(level = 1)